In [0]:
from pyspark.sql.functions import col, avg, stddev, count

# Nomes das nossas tabelas
raw_table = "default.raw_transactions"
core_table = "default.user_behavior_profiles"

raw_df = spark.read.table(raw_table)

In [0]:
# Calculando as métricas de perfil para cada usuário
user_profiles_df = raw_df.groupBy("user_id").agg(
    avg("amount_brl").alias("avg_amount"),
    stddev("amount_brl").alias("stddev_amount"),
    count("transaction_id").alias("transaction_count")
)

user_profiles_df = user_profiles_df.na.fill(0.0, subset=["stddev_amount"]) # Caso de 1 transação 
display(user_profiles_df)

In [0]:
from delta.tables import *

if spark.catalog.tableExists(core_table):
    print(f"Tabela '{core_table}' encontrada. Executando MERGE...")
    
    delta_table = DeltaTable.forName(spark, core_table)

    # Executando a operação MERGE (UPSERT)
    delta_table.alias("target") \
      .merge(
        user_profiles_df.alias("source"),
        "target.user_id = source.user_id"
      ) \
      .whenMatchedUpdateAll() \
      .whenNotMatchedInsertAll() \
      .execute()
    
    print(f"MERGE concluído com sucesso para a tabela '{core_table}'.")

else:
    # Se não existir, cria a tabela
    print(f"Tabela '{core_table}' não encontrada. Criando a tabela")
    user_profiles_df.write.format("delta").saveAsTable(core_table)
    
    print(f"Tabela '{core_table}' criada com sucesso.")

In [0]:
%sql SELECT * FROM default.user_behavior_profiles

In [0]:
%sql
DESCRIBE HISTORY default.user_behavior_profiles